# Remarks

* Data normalization
    * Mobilenet expects data from -1 to 1
        * Normalize Input Data or Include in Model
        * TFLite Conversion must fit according to decision
    * Ground Truth Data: for better inspection Data multiplied by 80. Undo the change in the Data Input Pipeline
* Overview in Tutorials:
    * tf.function
* Idea

# Libraries

In [1]:
#!for a in /sys/bus/pci/devices/*; do echo 0 | sudo tee -a $a/numa_node; done

import tensorflow as tf
import numpy as np
import os
import time
from datetime import datetime
#import sys
import matplotlib.pyplot as plt




import DataProcessing.data_processing as data_processing
import Nets.backbones as backbones
import Nets.features as features
import Nets.losses as losses
import Nets.metrics as metrics
import Nets.tools as tools


os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

#np.set_printoptions(threshold=sys.maxsize)

2022-05-03 10:33:56.934020: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-03 10:33:56.934080: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (david-ThinkPad-X1-Yoga-Gen-6): /proc/driver/nvidia/version does not exist
2022-05-03 10:33:56.944427: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Testing

# Options

In [2]:
DATA = 'SceneNet'
MODEL = 'SGED'
OUTPUT_TRAIN = 'outputTrain'
OUTPUT_TEST = 'outputTest'
HALF = True

IMG_SIZE_HEIGHT = 1280
IMG_SIZE_WIDTH = 720
NUM_CLASSES = 3
BATCH_SIZE = 3

#MAX_IMG_TRAIN = 1000
#MAX_IMG_TEST = 200
MAX_IMG_TRAIN = 1500
MAX_IMG_TEST = 300

CACHE = True
SEED = None
#SEED = 5

BACKBONE = "MobileNetV2"
output = 5
BACKBONE_OUTPUT = [1,2,3,output] #keep in mind: list start with 0.
BACKBONE_WEIGHTS = "imagenet"
ALPHA = 1.0
FINE_TUNING = True
TRAINABLE_OUTPUT_LAYER_IDX = 3 #(4-1) as indexint starts with 0



#EPOCHS = 30
EPOCHS = 10
fine_tune_epochs = 20

#Model Callback
MODEL_SAVE_EPOCH_FREQ = 5
DEL_CHECKPOINTS_OLD = True

# TENSORBOARD:
TENSORBOARD = False
DEL_OLD_TENSORBOARD = True

# SAVE
SAVE = True

# LOSS
WeightedMultiLabelSigmoidEdgeLoss = False
if WeightedMultiLabelSigmoidEdgeLoss:
    beta_upper = 0.95
    beta_lower = 0.05
    class_weighted = False

FocalLoss = True
if FocalLoss:
    gamma=2 
    alpha=1 
    weighted_beta=True 
    beta_upper=0.9 
    beta_lower=0.1
    class_weighted=False

THRESHOLD_EDGE_WIDTH = 1

#TESTING
test = False
if test:
    EPOCHS = 1
    MAX_IMG_TRAIN = 6
    MAX_IMG_TEST = 3

# Load Dataset, Preprocess Images and Dataset

In [3]:
tf.random.set_seed(SEED)

paths, files = data_processing.path_definitions(HALF, MODEL, DATA, OUTPUT_TRAIN, OUTPUT_TEST)

data_processing.clean_model_directories(paths, DEL_CHECKPOINTS_OLD, DEL_OLD_TENSORBOARD)

train_ds, img_count_train = data_processing.loader(paths,"TRAIN", IMG_SIZE_HEIGHT, IMG_SIZE_WIDTH, HALF, MAX_IMG_TRAIN)
train_ds = data_processing.dataset_processing(train_ds, CACHE=CACHE, SHUFFLE = True, BATCH_SIZE = BATCH_SIZE, PREFETCH = True, img_count = img_count_train)

test_ds, img_count_test = data_processing.loader(paths,"TEST", IMG_SIZE_HEIGHT, IMG_SIZE_WIDTH, HALF, MAX_IMG_TEST)
test_ds = data_processing.dataset_processing(test_ds, CACHE=CACHE, SHUFFLE = False, BATCH_SIZE = BATCH_SIZE, PREFETCH = True, img_count = img_count_test)

#print("Memory Usage corresponds to: ",tf.config.experimental.get_memory_info('GPU:0')['current'])

/home/david/SemesterProject/Datasets/SceneNet/outputTrain/half/images


TypeError: in user code:

    File "/home/david/SemesterProject/SGED/DataProcessing/data_processing.py", line 76, in parse_image  *
        image = tf.io.read_file(img_path)

    TypeError: Input 'filename' of 'ReadFile' Op has type float32 that does not match expected type of string.


# Model

In [ ]:
if HALF:
    output_dims = (int(IMG_SIZE_HEIGHT/2), int(IMG_SIZE_WIDTH/2))
else:
    output_dims = (IMG_SIZE_HEIGHT, IMG_SIZE_WIDTH)

# BACKBONE
backbone, output_names = backbones.get_backbone(name=BACKBONE,weights=BACKBONE_WEIGHTS,
                                          height=IMG_SIZE_HEIGHT,width=IMG_SIZE_WIDTH,
                                          alpha=ALPHA, output_layer = BACKBONE_OUTPUT, 
                                          trainable_idx=TRAINABLE_OUTPUT_IDX)

# DASPP
daspp = features.DASPP(layers[-1])

# Decoder
decoded = features.decoder(daspp, layers[1], output_dims=output_dims, NUM_CLASSES=NUM_CLASSES, num_side_filters = 6)


# SIDE FEATURES
# TODO: Upsampling: Nearest NEIGHBOUR ?
upsample_side_1 = features.side_feature_SGED(backbone.output[0], output_dims=output_dims ,interpolation="bilinear")
upsample_side_2 = features.side_feature_SGED(backbone.output[1], output_dims=output_dims ,interpolation="bilinear")
upsample_side_3 = features.side_feature_SGED(backbone.output[2], output_dims=output_dims ,interpolation="bilinear")

# TODO: adaptive weight fusion ?
# CONCATENATE
side_outputs = [upsample_side_1,upsample_side_2,upsample_side_3,decoded]
concat = features.shared_concatenation(side_outputs,NUM_CLASSES)

# FUSE_CLASSIFICATION
output = features.fused_classification(concat,NUM_CLASSES,name="output")

model = tf.keras.Model(inputs = backbone.input, outputs = output)

if WeightedMultiLabelSigmoidEdgeLoss:
    loss = lambda y_true, y_pred : losses.WeightedMultiLabelSigmoidLoss(y_true,y_pred,beta_lower=beta_lower,beta_upper=beta_upper, class_weighted=class_weighted)
elif FocalLoss:
    loss = lambda y_true, y_pred : losses.FocalLossEdges(y_true, y_pred, gamma=gamma, alpha=alpha, weighted_beta=weighted_beta,beta_lower=beta_lower,beta_upper=beta_upper, class_weighted=class_weighted)
else:
    raise ValueError("either FocalLoss or WeightedMultiLabelSigmoidLoss must be True")

#tf.keras.utils.plot_model(model,show_shapes = True,to_file = 'h.png')

# Compile and Train Model

In [ ]:
if TENSORBOARD:
    %load_ext tensorboard
    %tensorboard --logdir /home/david/SemesterProject/Models/SGED/logs

In [ ]:
# same Loss function is applied to both outputs, thus I only pass one loss function

# learning rate schedule
# base_learning_rate = 0.00125
base_learning_rate = 0.0015
end_learning_rate = 0.0001
decay_step = np.ceil(img_count_train / BATCH_SIZE)*EPOCHS
lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(base_learning_rate,decay_steps = decay_step,end_learning_rate = end_learning_rate, power = 0.9)

frequency = int(np.ceil(img_count_train / BATCH_SIZE)*MODEL_SAVE_EPOCH_FREQ)

logdir = os.path.join(paths['TBLOGS'], datetime.now().strftime("%Y%m%d-%H%M%S"))
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath = paths["CKPT"]+ "/ckpt-loss={loss:.2f}-epoch={epoch:.2f}",save_weights_only=False,save_best_only=False,monitor="val_loss",verbose=1,save_freq= frequency),
            tf.keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1)]

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=loss,
              metrics=[metrics.BinaryAccuracyEdges(threshold_prediction=0),
                       metrics.RecallEdges(threshold_prediction=0, threshold_edge_width=THRESHOLD_EDGE_WIDTH),
                       metrics.PrecisionEdges(threshold_prediction=0, threshold_edge_width=THRESHOLD_EDGE_WIDTH)])


history = model.fit(train_ds, epochs=EPOCHS, validation_data=test_ds, callbacks=callbacks)

# Plot Results

In [ ]:
plot_losses = ["loss"]
plot_metrics = ["accuracy_edges", "recall_edges", "precision_edges"]

path = os.path.join(paths["FIGURES"],"training.svg")

tools.plot_training_results(res=history.history, losses=plot_losses, metrics=plot_metrics, SAVE=SAVE, path=path)




In [ ]:
for img, label in test_ds.take(1):
    img, label = img, label
    
    predictions = model.predict(img)
    
predictions = tools.predict_class_postprocessing(predictions, threshold = 0.5)

path = os.path.join(paths["FIGURES"],"images")
tools.plot_images(images=img, labels=label, predictions=predictions,SAVE = SAVE, path = path, BS=3)

# Fine Tuning

In [ ]:
if FINE_TUNING:

    # Let's take a look to see how many layers are in the base model
    print("Number of layers in the base model: ", len(backbone.layers))

    # Fine-tune from this layer onwards
    fine_tune_output = output_names[2]

    backbone.trainable = True

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in backbone.layers:
        layer.trainable = False
        if layer.name == fine_tune_output:
            break
    
    
    total_epochs =  EPOCHS + fine_tune_epochs

    base_learning_rate = 0.0015
    end_learning_rate = 0.0005
    decay_step = np.floor(img_count_train / BATCH_SIZE)*fine_tune_epochs
    lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
        base_learning_rate,decay_steps = decay_step,end_learning_rate = end_learning_rate, power = 0.9)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                  loss=loss,
                  metrics=[metrics.BinaryAccuracyEdges(threshold_prediction=0),
                           metrics.RecallEdges(threshold_prediction=0, threshold_edge_width=THRESHOLD_EDGE_WIDTH),
                           metrics.PrecisionEdges(threshold_prediction=0, threshold_edge_width=THRESHOLD_EDGE_WIDTH)])

    history_fine = model.fit(train_ds, epochs=total_epochs, 
                               initial_epoch=history.epoch[-1],validation_data=train_ds.take(1), 
                               callbacks=callbacks)
    
    plot_losses = ["loss"]
    plot_metrics = ["accuracy_edges", "recall_edges", "precision_edges"]
    
    path = os.path.join(paths["FIGURES"],"fine_tuning_training.svg")
    
    tools.plot_training_results(res=history.history, res_fine = history_fine.history, 
                                losses=plot_losses, metrics=plot_metrics, SAVE=SAVE, path=path, EPOCHS = EPOCHS)

# Predictions

In [ ]:
for img, label in test_ds.take(1):
    img, label = img, label

    
predictions = model.predict(img)
predictions = tools.predict_class_postprocessing(predictions, threshold = 0.5)

path = os.path.join(paths["FIGURES"],"fine_tuning_images_0,5")
tools.plot_images(images=img, labels=label, predictions=predictions,SAVE = SAVE, path = path, BS=3)

# Save Model

In [4]:
if SAVE:
    model.save(paths["MODEL"])
    
    custom_objects = {"BinaryAccuracyEdges": metrics.BinaryAccuracy_Edges,
                      "RecallEdges": metrics.Recall_Edges,
                      "PrecisionEdges": metrics.Precision_Edges,
                      "lambda":loss}

NameError: name 'model' is not defined

# Predict Real Images

In [5]:
def predict_real(path,HEIGHT,WIDTH):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.expand_dims(image,axis = 0)
    image = tf.image.resize(image,(IMG_SIZE_HEIGHT, IMG_SIZE_WIDTH))
    pred = model.predict(image)
    
    val_max = tf.reduce_max(pred, axis = -1)
    idx_max = tf.argmax(pred, axis = -1) + 1

    predictions = tf.where(val_max >= 0, idx_max, 0)

    predictions = tf.expand_dims(predictions, axis = -1)

    return image,predictions


In [6]:
IMG_SIZE_HEIGHT = 1280
IMG_SIZE_WIDTH = 720

imgs = [1,2,3,4,5, 6]
path = '/home/david/test_images/'
Data = {'img':[],'mask':[]}

for i in imgs:
    img, mask = predict_real(path+str(i)+'.png',IMG_SIZE_HEIGHT,IMG_SIZE_WIDTH)
    Data['img'].append(img)
    Data['mask'].append(mask)


plt.figure(figsize=(10, 50))
size = len(Data['img'])
for k in range(size):
    plt.subplot(size,2, k*2+1)
    plt.title("Images")
    plt.imshow(tf.keras.preprocessing.image.array_to_img(Data['img'][k][0,:,:,:]))
    plt.subplot(size,2, k*2+2)
    plt.title("Estimation")
    plt.imshow(Data['mask'][k][0,:,:],cmap='gray')
    plt.axis('off')

if SAVE:
    plt.savefig(os.path.join(paths["FIGURES"],'real.svg'), bbox_inches='tight')
plt.show()

NameError: name 'model' is not defined

# Addtional Elements to Consider in other Projects

* Data augmentation for small datasets